In [1]:
state = 'AR' 
year = 2020
obj_type = 'perimeter'
starting_deviation = 0.01 

In [2]:
import sys, os
src_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(src_path)

In [3]:
filepath = '../../dat/' + str(year) + '/'
filename = state + '_county.json'
filename2 = state + '_county.shp'

In [4]:
from read import read_graph_from_json

G = read_graph_from_json(state, filepath + filename, year=year)
print(f"The state of {state} has {G._k} districts.")
G._ideal_population = sum(G.nodes[i]['TOTPOP'] for i in G.nodes) / G._k

The state of AR has 4 districts.


In [5]:
#import warm starts
sys.path.append(os.path.abspath('../heuristic'))

from AR_plans_2020 import plans
print(f"Loaded {len(plans)} plans from file.")
warm_starts = plans

Loaded 4993 plans from file.


In [ ]:
from epsilon_constraint import epsilon_constraint_method

(plans, obj_bounds, deviations) = epsilon_constraint_method(
            G,                 
            obj_type,          
            contiguity = 'lcut',                                             # {'lcut', 'scf', 'shir'} 
            cutoff=None,       
            verbose= True,
            warm_start_mode = 'user',                                        # {'None', 'user', 'refinement'}
            warm_starts=warm_starts,                                         # if you have user define warm starts else it is None
            starting_deviation=starting_deviation, 
            time_limit=7200, 
            sizes=None,      
            max_B=True,                                                      # If symmetry_breaking is 'orbitope' or you have warm_start, max_B should be True   
            symmetry_breaking='orbitope',                                    # {None, 'orbitope', 'rsum'} 
            state=state,
            year=year
        )

Initially, L = 745353 and U = 760409 and k = 4.

****************************************
Trying deviation = 7528.81
****************************************
Using user-provided warm starts.
Selected warm_start = [[4, 20, 28, 54, 61], [7, 17, 22, 23, 30, 50, 51, 52, 53, 70, 71], [0, 1, 6, 8, 9, 11, 18, 19, 21, 24, 25, 27, 29, 33, 34, 35, 36, 37, 40, 42, 45, 47, 48, 49, 55, 56, 57, 60, 64, 65, 66, 67, 72], [2, 3, 5, 10, 12, 13, 14, 15, 16, 26, 31, 32, 38, 39, 41, 43, 44, 46, 58, 59, 62, 63, 68, 69, 73, 74]]
Objective value: 45.27183118886377
Deviation: 4345.0

****************************************
Running labeling model!
****************************************
L = 745353 and U = 760409
Set parameter Username
Set parameter LicenseID to value 2608266
Academic license - for non-commercial use only - expires 2026-01-09
sizes =  [1, 1, 1, 1]
Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1


     0     0   30.47666    0  431   45.87739   30.47666  33.6%     -    2s
     0     0   30.50101    0  429   45.87739   30.50101  33.5%     -    2s
     0     0   30.50672    0  440   45.87739   30.50672  33.5%     -    2s
     0     0   30.50894    0  440   45.87739   30.50894  33.5%     -    2s
     0     0   30.56677    0  426   45.87739   30.56677  33.4%     -    2s
     0     0   30.57667    0  426   45.87739   30.57667  33.4%     -    2s
     0     0   30.57667    0  428   45.87739   30.57667  33.4%     -    2s
     0     0   30.57667    0  429   45.87739   30.57667  33.4%     -    2s
     0     0   30.60903    0  442   45.87739   30.60903  33.3%     -    2s
     0     0   30.61523    0  445   45.87739   30.61523  33.3%     -    2s
     0     0   30.61523    0  446   45.87739   30.61523  33.3%     -    2s
     0     0   30.61523    0  450   45.87739   30.61523  33.3%     -    2s
     0     0   30.61674    0  451   45.87739   30.61674  33.3%     -    2s
     0     0   30.63008  

  7938  1976   46.57284   26  501   49.23371   42.21962  14.2%   183   60s
  9465  2433   48.56075   26  339   49.23371   42.82771  13.0%   177   65s
 11289  2936   48.17710   28  455   49.23371   43.31189  12.0%   169   70s
 12919  3415   45.86765   29  385   49.23371   43.64933  11.3%   163   75s
 14123  3659     cutoff   27        49.23371   43.79099  11.1%   160   81s
 15497  4075 infeasible   28        49.23371   44.10357  10.4%   157   86s
 17674  4596   48.45735   30  398   49.23371   44.40444  9.81%   152   91s
 19284  4927   48.06736   34  302   49.23371   44.56536  9.48%   150   96s
 21035  5312   46.89390   26  444   49.23371   44.72373  9.16%   147  100s
 23116  5649     cutoff   28        49.23371   44.89155  8.82%   144  106s
 25740  6069   48.28445   29  480   49.23371   45.09107  8.41%   141  112s
*26547  5985              45      48.9264826   45.11596  7.79%   140  112s
 27156  6113   48.47611   32  232   48.92648   45.19896  7.62%   139  116s
 28695  6239   48.64038  

  5787  2485   51.36274   33  242   51.93963   41.34711  20.4%   151   45s
  8095  3197 infeasible   26        51.93963   42.89864  17.4%   145   50s
 10098  4168   46.42951   23  470   51.93963   43.42941  16.4%   141   55s
 12006  5113 infeasible   29        51.93963   43.85591  15.6%   140   60s
 13875  5913   51.50450   27  394   51.93963   44.12748  15.0%   137   65s
 15639  6837   48.51802   31  238   51.93963   44.35958  14.6%   137   70s
 18787  8309   50.60426   26  452   51.93963   44.59568  14.1%   134   76s
 20329  8967   51.36802   32  159   51.93963   44.72985  13.9%   133   80s
 23145 10225   47.02794   22  410   51.93963   44.91645  13.5%   132   86s
 24289 10719   48.72717   23  369   51.93963   44.97963  13.4%   132   90s
 26577 11848   50.15403   48   59   51.93963   45.10304  13.2%   132   95s
 31234 13725   49.15805   28  372   51.93963   45.30508  12.8%   130  100s
 36767 16187   51.00157   26  275   51.93963   45.51921  12.4%   129  106s
 41124 18143     cutoff  

In [ ]:
result = list(zip(plans, obj_bounds , deviations))

In [ ]:
no_solution_region = [0, min(round(r[2],1) for r in result)]
print(f"No feasible solution was found within the region: {no_solution_region}")

In [ ]:
from pareto import plot_pareto_frontiers

plot_pareto_frontiers(
                G,
                method='epsilon_constraint_method',
                plans=None,                                   #if method ='epsilon_constraint_method' is None 
                obj_types=obj_type,                               
                ideal_population=G._ideal_population,
                state=state,
                filepath=filepath,
                filename2=filename2,
                no_solution_region=no_solution_region,
                year=year,
                result=result                               #if method ='heuristic' is None 
             )

In [ ]:
from draw import draw_plan
from metrics import observed_deviation_persons, compute_obj

print(f"\n{'#' * 100}\nPareto maps for state {state}, objective {obj_type}\n{'#' * 100}\n")

format_obj = {
    'bottleneck_Polsby_Popper': lambda x: round(1 /x, 4),
    'cut_edges': lambda x: int(x)}
G._L = 0
G._U = G._k * G._ideal_population

for plan, obj_bound, dev in result:
    obs_dev = observed_deviation_persons(G, plan, G._ideal_population)
    obj = compute_obj(G, plan, obj_type)
    obj_val = format_obj.get(obj_type, lambda x: round(x, 4))(obj)
    deviation_percentage = round(100 * dev / G._ideal_population, 4)
    title = f"{round(obs_dev, 2)}-person deviation ({deviation_percentage}%), with {obj_val} {obj_type}"
    draw_plan(filepath, filename2, G, plan, title=title, year=year)